# Split Subjective Days

In [260]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import seaborn as sns
import pylab as py

# Read Spider FRP CSV file
summary_filename = 'Metazygia wittfeldae summary'
summary_df = pd.read_csv(frp_filename + '.csv')

# Extract FRP values for each spider
frp_df = summary_df.iloc[29:58, 5]
frp_values = frp_df.values.tolist()
#frp_values

In [272]:
summary_df = summary_df[(summary_df['Conditions'] == 'DD') & (summary_df['Spider ID'].str.match('Monitor 1'))]
frp_df = summary_df[['Spider ID', 'LombSc period']]

,Spider ID,LombSc period
29,Monitor 1 Spider 1,22.011150
30,Monitor 1 Spider 2,21.316001
31,Monitor 1 Spider 3,24.453386
32,Monitor 1 Spider 4,24.676897
33,Monitor 1 Spider 5,21.316001
34,Monitor 1 Spider 6,22.283665
35,Monitor 1 Spider 7,18.760847
36,Monitor 1 Spider 8,23.242881
37,Monitor 1 Spider 9,23.806503
38,Monitor 1 Spider 10,22.897901


# Filename

In [251]:
# Read spider activity CSV file

filename = 'Metazygia wittfeldae Monitor 1 Updated_DD'
df = pd.read_csv(filename + '.csv', index_col = 0)
df.index = pd.to_datetime(df.index)
#display(df)

# Split Subjective Days Function

Given arguments dataframe in DateTimeIndex, the initial day where the lights didn't turn on, and the FRP of the spider, returns the subjective days of the spider.

In [231]:
def split_subjective_days(df, initial_day, frp):
    # Get day where lights would turn on but they don't (DD)
    df_start = df.at_time('07:00:00')
    idx = df_start.index[0]
    
    # Subjective days starting from lights not being turned on to given FRP
    dti = pd.date_range(idx, periods = 10, freq = frp)
    dti = dti.round(freq = '1T')
    
    return dti

split_subjective_days(df, idx, str(frp_values[0]) + 'H')

DatetimeIndex(['2017-04-26 07:00:00', '2017-04-27 05:01:00',
               '2017-04-28 03:01:00', '2017-04-29 01:02:00',
               '2017-04-29 23:03:00', '2017-04-30 21:03:00',
               '2017-05-01 19:04:00', '2017-05-02 17:05:00',
               '2017-05-03 15:05:00', '2017-05-04 13:06:00'],
              dtype='datetime64[ns]', freq=None)

# Extracting Activity

In [256]:
def extract_activity_from_subjective_day(dti, start_index, end_index, column_name):
    # Extract activity based on subjective day index
    activitydf = df.loc[dti[start_index] : dti[end_index], [column_name]]
    #activitydf.to_csv(filename + '_subjective.csv')
    
    return activitydf

extract_activity_from_subjective_day(dti, 1, 2, 'Monitor 1 Spider 1')

,Monitor 1 Spider 1
Date-Time,
2017-04-27 05:01:00,0
2017-04-27 05:02:00,0
2017-04-27 05:03:00,0
2017-04-27 05:04:00,0
2017-04-27 05:05:00,0
...,...
2017-04-28 02:57:00,0
2017-04-28 02:58:00,0
2017-04-28 02:59:00,0
